In [1]:
import pandas as pd
from finvizfinance.screener.overview import Overview

def get_filtered_picks(countries=['Canada'], limit_per_country=500):
    all_stocks = []
    
    for country in countries:
        print(f"Fetching candidates for: {country}...")
        
        try:
            foverview = Overview()
            
            # --- STEP 1: Broaden the initial search ---
            filters_dict = {
                'Analyst Recom.': 'Buy or better',
                'Country': country
            }
            foverview.set_filter(filters_dict=filters_dict)
            
            # Get results
            df_results = foverview.screener_view()
            
            if not df_results.empty:
                df_results['Source_Country'] = country
                all_stocks.append(df_results)
            else:
                print(f"   No stocks found for {country}.")
                
        except Exception as e:
            print(f"   Error fetching {country}: {e}")

    if not all_stocks:
        print("\nNo stocks found from any country.")
        return pd.DataFrame()

    # Combine all results
    combined_df = pd.concat(all_stocks, ignore_index=True)
    
    # --- STEP 2: The Rating Filter ---
    cols = combined_df.columns.tolist()
    recom_col = 'Recom' if 'Recom' in cols else 'Analyst Recom'
    
    if recom_col in cols:
        # Convert column to numbers
        combined_df[recom_col] = pd.to_numeric(combined_df[recom_col], errors='coerce')
        
        # Filter: keep anything <= 2.0 (Buy or Strong Buy)
        print(f"Filtering out 'Hold' or worse (Rating > 2.0)...")
        combined_df = combined_df[combined_df[recom_col] <= 2.0 ] #use <=2.0 for normal output
        
        # Sort by best rating
        combined_df = combined_df.sort_values(by=recom_col, ascending=True)

    # Clean up columns for display
    desired_cols = ['Ticker', 'Company', 'Sector', 'Price', 'Source_Country', recom_col]
    final_cols = [c for c in desired_cols if c in combined_df.columns]
    
    return combined_df[final_cols].head(limit_per_country * len(countries))

# --- Usage ---
top_picks_canada = get_filtered_picks(['Canada'])

if not top_picks_canada.empty:
    # Ensure Price is numeric (so you can sort it yourself in the Data Viewer)
    top_picks_canada['Price'] = pd.to_numeric(top_picks_canada['Price'], errors='coerce')

    print(f"\nSuccess! Found {len(top_picks_canada)} 'Buy' rated Canadian stocks.")
    print("Variable 'top_picks' is ready. Click it in the Jupyter Variables view.")

else:
    print("DataFrame is empty.")

Fetching candidates for: Canada...
[Info] loading page [###########################---] 8/9 
Success! Found 162 'Buy' rated Canadian stocks.
Variable 'top_picks' is ready. Click it in the Jupyter Variables view.


C:\Users\jdcc3\AppData\Roaming\Python\Python313\site-packages\finvizfinance\screener\base.py:134: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([df, pd.DataFrame(frame)], ignore_index=True)


In [2]:
import pandas as pd
from finvizfinance.screener.overview import Overview

def get_filtered_picks(countries=['Canada'], limit_per_country=500):
    all_stocks = []
    
    for country in countries:
        print(f"Fetching candidates for: {country}...")
        
        try:
            foverview = Overview()
            
            # --- STEP 1: Broaden the initial search ---
            filters_dict = {
                'Analyst Recom.': 'Strong Buy (1)',
                'Country': country
            }
            foverview.set_filter(filters_dict=filters_dict)
            
            # Get results
            df_results = foverview.screener_view()
            
            if not df_results.empty:
                df_results['Source_Country'] = country
                all_stocks.append(df_results)
            else:
                print(f"   No stocks found for {country}.")
                
        except Exception as e:
            print(f"   Error fetching {country}: {e}")

    if not all_stocks:
        print("\nNo stocks found from any country.")
        return pd.DataFrame()

    # Combine all results
    combined_df = pd.concat(all_stocks, ignore_index=True)
    
    # --- STEP 2: The Rating Filter ---
    cols = combined_df.columns.tolist()
    recom_col = 'Recom' if 'Recom' in cols else 'Analyst Recom'
    
    if recom_col in cols:
        # Convert column to numbers
        combined_df[recom_col] = pd.to_numeric(combined_df[recom_col], errors='coerce')
        
        # Filter: keep anything <= 2.0 (Buy or Strong Buy)
        print(f"Filtering out 'Hold' or worse (Rating > 2.0)...")
        combined_df = combined_df[combined_df[recom_col] <= 2.0 ] #use <=2.0 for normal output
        
        # Sort by best rating
        combined_df = combined_df.sort_values(by=recom_col, ascending=True)

    # Clean up columns for display
    desired_cols = ['Ticker', 'Company', 'Sector', 'Price', 'Source_Country', recom_col]
    final_cols = [c for c in desired_cols if c in combined_df.columns]
    
    return combined_df[final_cols].head(limit_per_country * len(countries))

# --- Usage ---
Strong_Buy_canada = get_filtered_picks(['Canada'])

if not Strong_Buy_canada.empty:
    # Ensure Price is numeric (so you can sort it yourself in the Data Viewer)
    Strong_Buy_canada['Price'] = pd.to_numeric(Strong_Buy_canada['Price'], errors='coerce')

    print(f"\nSuccess! Found {len(Strong_Buy_canada)} ' Strong Buy' rated Canadian stocks.")
    print("Variable 'top_picks' is ready. Click it in the Jupyter Variables view.")

else:
    print("DataFrame is empty.")

Fetching candidates for: Canada...
[Info] loading page [########################------] 4/5 
Success! Found 94 ' Strong Buy' rated Canadian stocks.
Variable 'top_picks' is ready. Click it in the Jupyter Variables view.


In [3]:
import yfinance as yf
import pandas as pd
import numpy as np
import time

# --- CONFIGURATION ---
# Use the dataframe from your specific variable
source_df = Strong_Buy_canada 
input_variable_name = 'Strong_Buy_canada'
# ---------------------

if input_variable_name in locals() and not source_df.empty:
    tickers = source_df['Ticker'].unique().tolist()
    print(f"Total stocks to check: {len(tickers)}")
    
    # Settings
    chunk_size = 100 
    delay = 2        

    rsi_picks = []
    
    # Split tickers into chunks
    for i in range(0, len(tickers), chunk_size):
        chunk = tickers[i:i + chunk_size]
        print(f"Processing batch {i} to {i + len(chunk)}...")
        
        try:
            # Download data (US Listings -> USD Prices)
            data = yf.download(chunk, period="1y", interval="1d", progress=False, group_by='ticker', threads=True)
            
            if data is None or data.empty:
                continue

            for ticker in chunk:
                try:
                    # Handle MultiIndex structure
                    if len(chunk) > 1:
                        if ticker in data.columns.levels[0]:
                             df = data[ticker].copy()
                        else:
                            continue 
                    else:
                        df = data.copy()

                    df = df.dropna(subset=['Close'])

                    if len(df) < 200:
                        continue

                    # --- Calculations ---
                    ma_200 = df['Close'].rolling(window=200).mean().iloc[-1]
                    
                    delta = df['Close'].diff()
                    up = delta.clip(lower=0)
                    down = -1 * delta.clip(upper=0)
                    ma_up = up.ewm(com=13, adjust=False).mean()
                    ma_down = down.ewm(com=13, adjust=False).mean()
                    rs = ma_up / ma_down
                    rsi = 100 - (100 / (1 + rs))
                    
                    current_rsi = rsi.iloc[-1]
                    current_price = df['Close'].iloc[-1]

                    # --- Filter: RSI < 50 ---
                    if current_rsi < 50:
                        rsi_picks.append({
                            'Ticker': ticker,
                            'RSI': round(current_rsi, 2),
                            '200_Day_MA': round(ma_200, 2),
                            'Current_Price_USD': round(current_price, 2), # Explicitly USD
                            'Trend': 'Above 200MA' if current_price > ma_200 else 'Below 200MA'
                        })
                
                except Exception:
                    continue

        except Exception as e:
            print(f"  Error in batch starting at {i}: {e}")

        if i + chunk_size < len(tickers):
            time.sleep(delay)

    # --- FINAL PROCESSING ---
    if rsi_picks:
        rsi_df = pd.DataFrame(rsi_picks)
        
        # Merge technical data back into Strong_Buy_canada
        # We perform an 'inner' join, so only stocks that passed the RSI filter remain
        Strong_Buy_canada = pd.merge(source_df, rsi_df, on='Ticker', how='inner')
        
        # Sort by RSI (Most oversold first)
        Strong_Buy_canada = Strong_Buy_canada.sort_values(by='RSI')
        
        print(f"\nSuccess! Filtered {input_variable_name} down to {len(Strong_Buy_canada)} stocks with RSI < 50.")
        
        # Display results
        display_cols = ['Ticker', 'Company', 'Current_Price_USD', 'RSI', 'Trend', 'Recom']
        available_cols = [c for c in display_cols if c in Strong_Buy_canada.columns]
        display(Strong_Buy_canada[available_cols].head(10))
        
    else:
        print("No stocks passed the RSI < 50 filter.")
        Strong_Buy_canada = pd.DataFrame() 
else:
    print(f"No '{input_variable_name}' dataframe found. Please run the previous cell first.")

Total stocks to check: 94
Processing batch 0 to 94...


C:\Users\jdcc3\AppData\Local\Temp\ipykernel_19844\2035767958.py:29: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(chunk, period="1y", interval="1d", progress=False, group_by='ticker', threads=True)



Success! Filtered Strong_Buy_canada down to 32 stocks with RSI < 50.


,Ticker,Company,Current_Price_USD,RSI,Trend
4,DCBO,Docebo Inc,20.98,29.47,Below 200MA
10,FSI,Flexible Solutions International Inc,6.82,31.44,Above 200MA
24,PMN,ProMIS Neurosciences Inc,6.76,31.51,Below 200MA
27,STN,Stantec Inc,96.56,31.59,Below 200MA
15,INM,InMed Pharmaceuticals Inc,1.33,34.63,Below 200MA
13,HITI,High Tide Inc,2.61,35.45,Below 200MA
0,ABCL,AbCellera Biologics Inc,3.73,36.77,Above 200MA
7,EDSA,Edesa Biotech Inc,1.75,38.92,Below 200MA
18,LOOP,Loop Industries Inc,1.14,39.21,Below 200MA
3,BHST,BioHarvest Sciences Inc,6.73,39.43,Below 200MA


In [4]:
if not Strong_Buy_canada.empty:
    Strong_Buy_canada['Price'] = pd.to_numeric(Strong_Buy_canada['Price'], errors='coerce')
    
    mid_picks = Strong_Buy_canada[
        (Strong_Buy_canada['Price'] >= 5) & 
        (Strong_Buy_canada['Price'] < 30)
    ]
    
    print(f"\nFound {len(mid_picks)} stocks between $5 and $30 USD.")
    if not mid_picks.empty:
        display(mid_picks[['Ticker', 'Price', 'RSI', 'Trend']].head())
else:
    print("DataFrame is empty.")


Found 7 stocks between $5 and $30 USD.


,Ticker,Price,RSI,Trend
4,DCBO,20.98,29.47,Below 200MA
10,FSI,6.82,31.44,Above 200MA
24,PMN,6.76,31.51,Below 200MA
3,BHST,6.73,39.43,Below 200MA
28,USGO,9.87,43.66,Below 200MA


In [5]:
if not Strong_Buy_canada.empty:
    # Ensure Price is numeric
    Strong_Buy_canada['Price'] = pd.to_numeric(Strong_Buy_canada['Price'], errors='coerce')
    
    # Filter
    cheap_picks = Strong_Buy_canada[Strong_Buy_canada['Price'] < 5]
    
    print(f"\nFound {len(cheap_picks)} stocks less than $5 USD.")
    if not cheap_picks.empty:
        display(cheap_picks[['Ticker', 'Price', 'RSI', 'Trend']].head())
else:
    print("DataFrame is empty.")


Found 21 stocks less than $5 USD.


,Ticker,Price,RSI,Trend
15,INM,1.33,34.63,Below 200MA
13,HITI,2.61,35.45,Below 200MA
0,ABCL,3.73,36.77,Above 200MA
7,EDSA,1.75,38.92,Below 200MA
18,LOOP,1.14,39.21,Below 200MA


In [6]:
if not Strong_Buy_canada.empty:
    Strong_Buy_canada['Price'] = pd.to_numeric(Strong_Buy_canada['Price'], errors='coerce')
    
    Main_TFSA_picks = Strong_Buy_canada[Strong_Buy_canada['Price'] > 20]
    
    print(f"\nFound {len(Main_TFSA_picks)} stocks greater than $20 USD.")
    if not Main_TFSA_picks.empty:
        display(Main_TFSA_picks[['Ticker', 'Price', 'RSI', 'Trend']].head())
else:
    print("DataFrame is empty.")


Found 5 stocks greater than $20 USD.


,Ticker,Price,RSI,Trend
4,DCBO,20.98,29.47,Below 200MA
27,STN,96.56,31.59,Below 200MA
12,GIL,55.82,42.08,Above 200MA
11,FSV,156.85,44.57,Below 200MA
20,MEOH,35.61,48.66,Above 200MA


In [7]:
"""
import yfinance as yf
import time

# Loop through EVERY symbol in the list (removed the [:3])
for symbol in ticker_list:
    
    # 1. Fetch data
    stock = yf.Ticker(symbol)
    print(f"\n--- Checking {symbol} ---")
    
    try:
        # 2. Get the upgrades/downgrades
        upgrades = stock.upgrades_downgrades
        
        if not upgrades.empty:
            # Filter for 2025 actions using the .index fix
            recent = upgrades[upgrades.index >= '2025-11-01']
            
            if not recent.empty:
                print(recent)
            else:
                print(f"No actions for {symbol} in 2025.")
        else:
            print(f"No analyst history found for {symbol}.")
            
    except Exception as e:
        print(f"Error fetching {symbol}: {e}")

    # 3. CRITICAL: Sleep for 1 second to avoid getting banned
    time.sleep(1)
"""

'\nimport yfinance as yf\nimport time\n\n# Loop through EVERY symbol in the list (removed the [:3])\nfor symbol in ticker_list:\n    \n    # 1. Fetch data\n    stock = yf.Ticker(symbol)\n    print(f"\n--- Checking {symbol} ---")\n    \n    try:\n        # 2. Get the upgrades/downgrades\n        upgrades = stock.upgrades_downgrades\n        \n        if not upgrades.empty:\n            # Filter for 2025 actions using the .index fix\n            recent = upgrades[upgrades.index >= \'2025-11-01\']\n            \n            if not recent.empty:\n                print(recent)\n            else:\n                print(f"No actions for {symbol} in 2025.")\n        else:\n            print(f"No analyst history found for {symbol}.")\n            \n    except Exception as e:\n        print(f"Error fetching {symbol}: {e}")\n\n    # 3. CRITICAL: Sleep for 1 second to avoid getting banned\n    time.sleep(1)\n'